## Real-time processing of Air Quality Data for Anomaly detection


To integrate our Python program with Bytewax for building a dataflow architecture, where input and deserialization are stateless and anomaly detection is stateful, we'll follow these steps:

1. Set up the Bytewax Dataflow: Define the dataflow to ingest data, perform the deserialization, imputation, and pass the data through the anomaly detection which is stateful.
2. Integrate Stateless Steps: These include reading input, deserializing data, and imputing missing values using KNN.
3. Integrate Stateful Step: This will be your anomaly detection, which maintains state over the window of data it analyzes.

In [2]:
!pip install bytewax==0.19 python-dotenv scipy==1.13.0 kafka-python==2.0.2 --q
!pip install pandas==2.0.3 river --q
!pip install scikit-learn==1.4.2 --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.5/246.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.8/796.8 kB 16.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 21.9 MB/s eta 0:00:00


In [2]:
import bytewax.operators as op

from bytewax.dataflow import Dataflow
from bytewax.testing import TestingSource
from bytewax.connectors.stdio import StdOutSink

from bytewax.inputs import StatelessSourcePartition, DynamicSource


import json
from bytewax.testing import run_main

import requests
import json
from datetime import datetime, timezone
from river import anomaly
from sklearn.impute import KNNImputer
import pandas as pd
import numpy as np


In [3]:
# Opening JSON file
f = open('data.json')

# returns JSON object as
# a dictionary
data = json.load(f)



To convert the `serialize` function into a Bytewax stream-equivalent format, we need to create a data source that behaves as a generator or a source of streaming data. Below, I will define two classes to model this behavior: one for partition-specific streaming data (`SerializedData`), and another to encapsulate the dynamic data generation across potentially multiple workers (`SerializedInput`).

Step 1: Define `SerializedData` as a `StatelessSourcePartition`
This class will act as a source partition that iterates over a dataset, serializing each entry according to the provided headers and fields.

Step 2: Define `SerializedInput` as a `DynamicSource`
This class encapsulates the partition management for the data source, ensuring that each worker in a distributed environment gets a proper instance of the source partition.

In [4]:
class SerializedData(StatelessSourcePartition):
    """
    Emit serialized data directly for simplicity. This class will serialize
    each entry in the 'data' list by mapping it to the corresponding 'fields'.
    """
    def __init__(self, full_data):
        self.fields = full_data['fields']
        self.data_entries = full_data['data']
        self.metadata = {k: v for k, v in full_data.items() if k not in ['fields', 'data']}
        self._it = iter(self.data_entries)

    def next_batch(self):
        try:
            data_entry = next(self._it)
            # Map each entry in 'data' with the corresponding field in 'fields'
            data_dict = dict(zip(self.fields, data_entry))
            # Merge metadata with data_dict to form the complete record
            complete_record = {**self.metadata, **{"data": data_dict}}
            # Serialize the complete record
            serialized = json.dumps(complete_record).encode('utf-8')
            return [serialized]
        except StopIteration:
            raise StopIteration


class SerializedInput(DynamicSource):
    """
    Dynamic data source that partitions the input data among workers.
    """
    def __init__(self, data):
        self.data = data
        self.total_entries = len(data['data'])

    def build(self, step_id, worker_index, worker_count):
        # Calculate the slice of data each worker should handle
        part_size = self.total_entries // worker_count
        start = part_size * worker_index
        end = start + part_size if worker_index != worker_count - 1 else self.total_entries

        # Create a partition of the data for the specific worker
        # Note: This partitions only the 'data' array. Metadata and fields are assumed
        # to be common and small enough to be replicated across workers.
        data_partition = {
            "api_version": self.data['api_version'],
            "time_stamp": self.data['time_stamp'],
            "data_time_stamp": self.data['data_time_stamp'],
            "max_age": self.data['max_age'],
            "firmware_default_version": self.data['firmware_default_version'],
            "fields": self.data['fields'],
            "data": self.data['data'][start:end]
        }

        return SerializedData(data_partition)

* Data Initialization: The `SerializedData` class now takes the entire data structure, keeps the metadata, and iterates over the data list. Each entry in data is mapped to the corresponding field specified in fields, combined with the metadata, serialized into a JSON string, and then encoded.

* Integration into Dataflow: The class is used directly within a Bytewax dataflow as an input source, demonstrating how serialized data would be produced from the structured input.

We can then deserialize the data with a simple function.


In [5]:
def process_deserialized_data(byte_data):
    """Deserialize byte data and prepare for stateful processing."""
    sensor_data = json.loads(byte_data.decode('utf-8'))['data']
    key = str(sensor_data.get("sensor_index", "default"))
    return (key, sensor_data)

Next we can perform imputation of the missing values.

Should KNN Imputation Be Stateful or Stateless?

Stateless processing implies that each data item is processed independently without any need to remember past interactions. This is typically not the case with KNN imputation:

* Stateful: The KNN algorithm typically benefits from "remembering" the dataset it uses to predict missing values because it bases its imputation on the k-nearest neighbors. Therefore, a stateful approach is often necessary if you need to continuously update the training dataset as new data arrives or if the dataset itself is too large to handle in a single batch efficiently.

* Stateless: If the data chunks you process are independent or if the dataset can be managed in small batches without the need for continuity between batches, you could consider a stateless approach.

For Bytewax, since it’s primarily built for stream processing, we might need to adjust your KNN implementation to fit into a stateful paradigm if our dataset is dynamically growing or if you need the model to adapt continuously as new data arrives.



In [16]:
def impute_data_with_knn(batch):
    try:
        df = pd.DataFrame(batch[1])

        # Convert all columns to numeric, coercing errors where data is non-numeric
        df = df.apply(pd.to_numeric, errors='coerce')

        # Debugging output to check DataFrame structure before imputation
        print(df.head())

        # Select numeric columns only for imputation
        numeric_columns = df.select_dtypes(include=[np.number]).columns
        imputer = KNNImputer(n_neighbors=5, weights='uniform')

        # Fit and transform the data
        print("Applying KNN Imputer...")
        imputed_array = imputer.fit_transform(df[numeric_columns])

        # Ensure the output is correctly sized
        if imputed_array.shape[1] == len(numeric_columns):
            df[numeric_columns] = imputed_array
        else:
            raise ValueError("Mismatch in column sizes after imputation.")

        # Debugging output to check DataFrame after imputation
        print("DataFrame after imputation:")
        print(df.info())

        return df.to_dict(orient='records')

    except Exception as e:
        print(f"Error during imputation: {e}")
        return None

Step 2: Integrate `KNNImputation` into the Bytewax Dataflow
Next, use this class in your dataflow. Since we're focusing on processing rather than generating new data from a source, we'll integrate it directly after data deserialization:

In [7]:
from bytewax.operators import window as wop

from datetime import datetime, timedelta, timezone


In [8]:
def extract_data_from_window(windowed_data):
    """Extract only the sensor data from the windowed output, ignoring metadata."""
    return [item[1] for item in windowed_data]  # Adjust this based on the actual structure of the windowed data


In [17]:
# Setup the dataflow
flow = Dataflow("air-quality-flow")
inp = op.input("inp", flow, SerializedInput(data))
deserialize = op.map("deserialize", inp, process_deserialized_data)

windower = wop.TumblingWindow(
    length=timedelta(seconds=10),
    align_to=datetime(2023, 1, 1, tzinfo=timezone.utc)
)
clock = wop.SystemClockConfig()
window = wop.collect_window("windowed_data", deserialize, clock, windower)

# Map to extract data from the window
extracted = op.map("extract_data", window, extract_data_from_window)

# # # Impute data
impute = op.map("impute", extracted, impute_data_with_knn)

# Output or further processing
op.inspect("inspect_impute", impute)
run_main(flow)

Streaming output truncated to the last 5000 lines.
 4   latitude      1 non-null      float64
 5   longitude     1 non-null      float64
 6   humidity      1 non-null      float64
 7   temperature   1 non-null      float64
 8   pressure      1 non-null      float64
 9   pm1.0         1 non-null      float64
 10  pm2.5_alt     1 non-null      float64
 11  pm10.0        1 non-null      float64
 12  pm1.0_cf_1    1 non-null      float64
 13  pm2.5_atm     1 non-null      float64
 14  pm2.5_cf_1    1 non-null      float64
 15  pm10.0_cf_1   1 non-null      float64
dtypes: float64(16)
memory usage: 256.0 bytes
None
   sensor_index  date_created  rssi  uptime  latitude  longitude  humidity  \
0         10182    1525379876   -63   34533  48.29451 -120.03911        20   

   temperature  pressure  pm1.0  pm2.5_alt  pm10.0  pm1.0_cf_1  pm2.5_atm  \
0           68    958.75    0.0        0.3     0.1         0.0        0.1   

   pm2.5_cf_1  pm10.0_cf_1  
0         0.1          0.1  
Applying KNN

KeyboardInterrupt: (src/run.rs:150:17) interrupt signal received, all processes have been shut down